In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests
import random
import datetime
from urllib.parse import urlparse

#Redirects
#r = requests.get('http://github.com', allow_redirects=True)

random.seed(0.112)
URL = 'https://kaledinaoksana.github.io/portfolio.github.io/'
#URL = 'http://en.wikipedia.org/wiki/Kevin_Bacon'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36", 
           "Accept-Encoding":"gzip, deflate, br", 
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
           "DNT":"1",
           "Connection":"close", 
           "Upgrade-Insecure-Requests":"1"
          }

def getLink(URL):
    page = requests.get(URL,headers=headers)
    bs = BeautifulSoup(page.content, 'html.parser')
    #print(bs)
    #return bs.find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    return bs.find_all('a')
 
      
#pattern = re.compile('[*#*]')
pattern = re.compile('[*#//*]') 

for link in getLink(URL):
    if 'href' in link.attrs:
        if (pattern.search(link.attrs['href']) is None):
            print(link.attrs['href'])
        


index.html
crypto.html
kpmg.html
datacleaning.html
amazon.html
covid.html


In [4]:
#Retrieves a list of all Internal links found on a page
def getInternalLinks(bs,includeUrl):
    global mylink
    pattern = re.compile('[*#//*]') 
    internalLinks = []
    links = bs.find_all('a')
    for link in links:
        if 'href' in link.attrs:
            if (pattern.search(link.attrs['href']) is None):
                if ( includeUrl != (mylink+'/'+ link.attrs['href'])):
                    if link.attrs['href'] not in internalLinks:
                        internalLinks.append(includeUrl+link.attrs['href'])

    return internalLinks


#Retrieves a list of all external links found on a page
def getExternalLinks(bs, excludeUrl):
    externalLinks = []
    #Finds all links that start with "http" or "www" that do not contain the current URL
    for link in bs.find_all('a', href=re.compile('^(http|www)((?!'+excludeUrl+').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    try:
        html = urlopen(startingPage)
        bs = BeautifulSoup(html, 'html.parser')
    except:
        page = requests.get(startingPage,headers=headers, allow_redirects=True)
        bs = BeautifulSoup(page.content, 'html.parser')
    
    externalLinks = getExternalLinks(bs,urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print('No external links, looking around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme,urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs,startingPage)
        return getRandomExternalLink(internalLinks[random.randint(0,len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]

def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    followExternalOnly(externalLink)
    


In [1]:


# Collects a list of all external URLs found on the site
allExtLinks = set()
allIntLinks = set()

#mylink = 'https://kaledinaoksana.github.io/portfolio.github.io/'
mylink = 'https://kaledinaoksana.github.io/portfolio.github.io/index.html'

def getAllExternalLinks(siteUrl):
    
    html = urlopen(siteUrl)
    domain = 'https://kaledinaoksana.github.io/portfolio.github.io/'
    bs = BeautifulSoup(html, 'html.parser')
    
        
    internalLinks = getInternalLinks(bs, domain)
    externalLinks = getExternalLinks(bs, domain)

    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            getAllExternalLinks(link)

allIntLinks.add(mylink)
getAllExternalLinks(mylink)

print('INT:')
for link in allIntLinks:
    print(link)
            
print('EXT:')           
for link in allExtLinks:
    print(link)
       


NameError: name 'urlopen' is not defined